# AAI540 - Module 2 Assignment

Victor Hugo Germano

## Initializing Dependencies

In [2]:
%store -r setup_dependencies_passed

In [3]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

print(setup_dependencies_passed)

True


In [4]:
%store -r setup_s3_bucket_passed

In [5]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [6]:
print(setup_s3_bucket_passed)

True


In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Creating Bucket and Loading file

In [11]:
# define s3 address
s3_private_path_data = "s3://{}/music-artists/csv".format(bucket)
print(s3_private_path_data)

s3://sagemaker-us-east-1-440542329720/music-artists/csv


In [12]:
%store s3_private_path_data

Stored 's3_private_path_data' (str)


In [25]:

local_file_path = 'data/dataset.csv'
s3_object_key = 'dataset-artists-m2.csv'

%store local_file_path
%store s3_object_key

Stored 'local_file_path' (str)
Stored 's3_object_key' (str)


In [28]:
# copy csv to s3
!aws s3 cp $local_file_path $s3_private_path_data/$s3_object_key 


upload: data/dataset.csv to s3://sagemaker-us-east-1-440542329720/music-artists/csv/dataset-artists-m2.csv


In [29]:
print(s3_private_path_data)

s3://sagemaker-us-east-1-440542329720/music-artists/csv


In [30]:
!aws s3 ls $s3_private_path_data/

2026-01-14 20:52:20   20118244 dataset-artists-m2.csv


## ATHENA Setup

In [31]:
ingest_create_athena_db_passed = False

In [3]:
from pyathena import connect
import awswrangler as wr
import warnings
warnings.filterwarnings('ignore')

database_name = "artistm2"

In [4]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS artistm2


In [5]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [6]:
# Verify execution
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,artistm2
1,default
2,dsoaws


In [8]:
table_name = "artist_m2"



In [36]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


## CREATE TABLE FROM CSV

In [13]:

# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        line_id int,
        track_id string,
        artists string,
        album_name string,
        track_name string,
        popularity int,
        duration_ms int,
        explicit boolean,
        danceability float,
        energy float,
        key float,
        loudness float,
        mode float,
        speechiness float,
        acousticness float,
        instrumentalness float,
        liveness float,
        valence float,
        tempo float,
        time_signature float,
        track_genre string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
)
STORED AS TEXTFILE
LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')
""".format(
    database_name, table_name, s3_private_path_data
)

print(statement)

# TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')
# ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' 

CREATE EXTERNAL TABLE IF NOT EXISTS artistm2.artist_m2(
        line_id int,
        track_id string,
        artists string,
        album_name string,
        track_name string,
        popularity int,
        duration_ms int,
        explicit boolean,
        danceability float,
        energy float,
        key float,
        loudness float,
        mode float,
        speechiness float,
        acousticness float,
        instrumentalness float,
        liveness float,
        valence float,
        tempo float,
        time_signature float,
        track_genre string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
)
STORED AS TEXTFILE
LOCATION 's3://sagemaker-us-east-1-440542329720/music-artists/csv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')



In [14]:
#statement = "DROP TABLE {}.{}".format(database_name, table_name)
pd.read_sql(statement, conn)

""


In [15]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,artist_m2


## Querying the Database

In [139]:
statement = """
    SELECT
    track_name,
    energy
    FROM {}.{}
    WHERE energy >= 0.5
    LIMIT 10
    """.format(database_name, table_name)
df = pd.read_sql(statement, conn)
print(df)

                                          track_name  energy
0                                             Hunger   0.632
1                                    Hold On - Remix   0.780
2                   Falling in Love at a Coffee Shop   0.561
3                                           Unlonely   0.667
4                                          The Haves   0.519
5                             You and Me on the Rock   0.686
6                             You and Me on the Rock   0.686
7  Speak Your Mind (From the Netflix Series "We T...   0.666
8                             You and Me on the Rock   0.686
9                             You and Me on the Rock   0.686


In [140]:
df = df[df['energy'] >= 0.5]
df = df[['track_name', 'energy']]
print(df)

                                          track_name  energy
0                                             Hunger   0.632
1                                    Hold On - Remix   0.780
2                   Falling in Love at a Coffee Shop   0.561
3                                           Unlonely   0.667
4                                          The Haves   0.519
5                             You and Me on the Rock   0.686
6                             You and Me on the Rock   0.686
7  Speak Your Mind (From the Netflix Series "We T...   0.666
8                             You and Me on the Rock   0.686
9                             You and Me on the Rock   0.686


## Querying the database

In [141]:
# List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99
statement = """
    SELECT
    artists,
    album_name,
    track_name, 
    popularity
    FROM {}.{}
    WHERE popularity >= 99
    LIMIT 10
    """.format(database_name, table_name)
df = pd.read_sql(statement, conn)
print(df)


                artists                             album_name  \
0  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)   
1      Bizarrap;Quevedo  Quevedo: Bzrp Music Sessions, Vol. 52   
2  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)   

                              track_name  popularity  
0              Unholy (feat. Kim Petras)         100  
1  Quevedo: Bzrp Music Sessions, Vol. 52          99  
2              Unholy (feat. Kim Petras)         100  


In [21]:
# List artists with an average popularity of 92
statement = """
    SELECT artists,
        AVG(popularity) AS avg_popularity
    FROM {}.{}
    GROUP BY artists
    HAVING AVG(popularity) = 92
    ORDER BY artists
    LIMIT 10;
    """.format(database_name, table_name)
df = pd.read_sql(statement, conn)
print(df)

             artists  avg_popularity
0       Harry Styles            92.0
1  Rema;Selena Gomez            92.0


In [19]:
# List the Top 10 genres with the highest average energy

statement = """
    SELECT track_genre, AVG(energy) AS avg_energy
    FROM {}.{}
    WHERE track_genre is not NULL
    GROUP BY track_genre
    ORDER BY avg_energy DESC
    limit 10
    """.format(database_name, table_name)
df = pd.read_sql(statement, conn)


print(df)


     track_genre  avg_energy
0    death-metal    0.931470
1      grindcore    0.924201
2      metalcore    0.914485
3          happy    0.910971
4      hardstyle    0.901246
5  drum-and-bass    0.876635
6    black-metal    0.874897
7    heavy-metal    0.874003
8          party    0.871237
9         j-idol    0.868677


In [144]:
# How many tracks is Bad Bunny on?

statement = """
    SELECT count(track_name) as songs_bb
    FROM {}.{}
    WHERE artists LIKE '%Bad Bunny%'
    """.format(database_name, table_name)

df = pd.read_sql(statement, conn)
print(df)

   songs_bb
0       416


In [145]:
# Show the top 10 genres in terms of popularity, sorted by their most popular track

statement = """
SELECT
    track_genre,
    track_name,
    popularity
FROM (
    SELECT
        track_genre,
        track_name,
        popularity,
        ROW_NUMBER() OVER (
            PARTITION BY track_genre
            ORDER BY popularity DESC
        ) AS rn
    FROM {}.{}
    WHERE track_genre IS NOT NULL
) t
WHERE rn = 1
ORDER BY popularity DESC
LIMIT 10;
""".format(database_name, table_name)

df = pd.read_sql(statement, conn)
print(df)


  track_genre                             track_name  popularity
0         pop              Unholy (feat. Kim Petras)         100
1       dance              Unholy (feat. Kim Petras)         100
2     hip-hop  Quevedo: Bzrp Music Sessions, Vol. 52          99
3       latin                             La Bachata          98
4      reggae                             La Bachata          98
5         edm                        I'm Good (Blue)          98
6   reggaeton                             La Bachata          98
7      latino                             La Bachata          98
8        rock                        I Ain't Worried          96
9       piano                        I Ain't Worried          96


In [1]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>